In [1]:
#### 컬러 히스토그램
import cv2 as cv
import numpy as np

src = cv.imread("test.jpg", cv.IMREAD_COLOR)
bgr_planes = cv.split(src) ## rgb 채널로 분리

histSize = 256 ## 픽셀값이 0 ~ 255 
histRange = (0,256) ## 0 ~ 255 (256은 포함 x)

accumulate = False ## 히스토그램 막대크기가 똑같고 처음 시작시 히스토그램이 비어있도록

## B, G, R 채널 별로 히스토그램을 계산
b_hist = cv.calcHist(bgr_planes,[0],None,[histSize],histRange,accumulate=accumulate) ## 파란색
g_hist = cv.calcHist(bgr_planes,[1],None,[histSize],histRange,accumulate=accumulate) ## 초록색
r_hist = cv.calcHist(bgr_planes,[2],None,[histSize],histRange,accumulate=accumulate) ## 빨간색

## 히스토오그램을 보여줄 이미지를 생성
hist_w = 256*3
hist_h = 400
histImage = np.zeros((hist_h,hist_w,3),dtype=np.uint8) ## 높이,널비,3개,타입은 unit8

## 히스토그램을 정규화
cv.normalize(b_hist,b_hist,alpha=0,beta=hist_h,norm_type=cv.NORM_MINMAX)
cv.normalize(g_hist,g_hist,alpha=0,beta=hist_h,norm_type=cv.NORM_MINMAX)
cv.normalize(r_hist,r_hist,alpha=0,beta=hist_h,norm_type=cv.NORM_MINMAX)

## 히스토그램을 그리기
for i in range(0,histSize):
    cv.line(histImage,(i,hist_h-int(np.round(b_hist[i]))),(i,hist_h-0),(255,0,0),thickness=2) ## 파란색
    cv.line(histImage, (i+256,hist_h-int(np.round(g_hist[i-1]))),(i+256,hist_h-0),(0,255,0),thickness=2) ## 초록색
    cv.line(histImage,(i+256*2,hist_h-int(np.round(r_hist[i-1]))),(i+256*2,hist_h-0),(0,0,255),thickness=2) ## 빨간색
    
## 결과화면 보여주기
cv.imshow('Source Image', src)
cv.imshow('Histogram', histImage)
cv.waitKey()

-1

In [2]:
#### 흑백 히스토그램
import cv2 as cv
import numpy as np

src= cv.imread("test.jpg", cv.IMREAD_GRAYSCALE) ## 흑백으로 불러오기

histSize = 256 ## 픽셀값이 0 ~ 255 
histRange = (0,256) ## 0 ~ 255 (256은 포함 x)
accumulate = False ## 히스토그램 막대크기가 똑같고 처음 시작시 히스토그램이 비어있도록

## 히스토그램 계산
gray_hist = cv.calcHist([src], [0] ,None, [histSize], histRange, accumulate=accumulate) 

## 히스토그램을 보여줄 이미지 생성
hist_w = 256
host_h = 400
histImage = np.zeros((hist_h,hist_w,1),dtype=np.uint8) ## 1개 그래프

## 히스토그램 정규화
cv.normalize(gray_hist, gray_hist, alpha=0, beta=hist_h, norm_type=cv.NORM_MINMAX)

## 히스토그램 그리기
for i in range(0,histSize) :
    cv.line(histImage,(i,hist_h-int(np.round(gray_hist[i]))), (i,hist_h-0), (255,255,255), thickness=2)
    
## 결과화면 보여주기
cv.imshow('Source Image', src)
cv.imshow('Histogram', histImage)
cv.waitKey()

-1

In [ ]:
## 히스토그램 평활화
import cv2 as cv
import numpy as np

def draw_histogram(img):
    hist_h = img.shape[0]
    hist_w = 256
    img_histogram1 = np.zeros((hist_h,hist_w,1), dtype=np.uint8)
    
    ## 히스토그램 계산
    histSize = 256 ## 픽셀값이 0 ~ 255 
    histRange = (0,256) ## 0 ~ 255 (256은 포함 x)
    accumulate = False ## 히스토그램 막대크기가 똑같고 처음 시작시 히스토그램이 비어있도록
    hist_item = cv.calcHist([img],[0],None,[histSize],histRange,accumulate=accumulate)

    cv.normalize(hist_item, hist_item, alpha=0, beta=hist_h, norm_type=cv.NORM_MINMAX)

    for i in range(0,histSize):
        cv.line(img_histogram1,(i,hist_h-int(np.round(hist_item[i]))), (i,hist_h), (255,255,255))

    ## 누적 히스토그램을 그릴 이미지 생성
    img_histogram2 = np.zeros((hist_h,hist_w,1), dtype=np.uint8)
    
    ## 누적 히스토그램을 계산하여 그리기
    c_hist = np.zeros(hist_item.shape, dtype=hist_item.dtype)
    
    c_hist[0] = hist_item[0]
    for i in range(1,hist_item.shape[0]):
        c_hist[i] = hist_item[i] + c_hist[i-1]
        
    cv.normalize(c_hist, c_hist, alpha=0, beta=hist_h, norm_type=cv.NORM_MINMAX)
    
    for i in range(1,histSize):
        pts = []
        pts.append((i,hist_h-np.round(c_hist[i])))
        pts.append(((i-1),hist_h-np.round(c_hist[i-1])))
        pts = np.array(pts)
        pts = np.int32(pts)
        cv.polylines(img_histogram2,[pts],False,(255,255,255))
        
    result = cv.hconcat((img_histogram1,img_histogram2))
    
    return result


img_gray = cv.imread("test.jpg", 0)
img_histo1 = draw_histogram(img_gray)

result1 = cv.hconcat((img_gray, img_histo1))

cv.imshow("restul1", result1)

## 히스토그램 평활화 적용후 히스토그램
img_equ = cv.equalizeHist(img_gray)
img_histo2 = draw_histogram(img_equ)
result2 = cv.hconcat((img_equ, img_histo2))
cv.imshow("result2", result2)
cv.waitKey(0)
cv.destroyAllWindows()

In [1]:
## 컬러 -> 흑백
import cv2

img = cv2.imread("test.jpg")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2.imshow("img", img)
cv2.imshow("img_gray", img_gray)
cv2.waitKey()

-1